In [1]:
import os
import requests
from pydantic import BaseModel, ValidationError
from typing import Annotated, Literal, Optional
from datetime import datetime
from autogen import ConversableAgent, register_function
from dotenv import load_dotenv
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.chains import LLMChain
import numpy as np
import pandas as pd
from io import StringIO
import spacy
from bs4 import BeautifulSoup
import time
import json
import re
from typing import List, Dict, Annotated

load_dotenv()


True

In [2]:
from openai import OpenAI
import os
import json
import re
import pdfkit
import logging
from datetime import datetime

try:
    from urllib.request import urlopen
except ImportError:
    from urllib2 import urlopen

import certifi

# logging.basicConfig(level=logging.INFO)

In [3]:
FMP_API_KEY = os.getenv("FMP_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [4]:
class PromptInput(BaseModel):
    prompt: Annotated[str, "Input query or prompt"]

class DocumentSelectorOutput(BaseModel):
    ticker: Annotated[str, "The ticker symbol related to user query (e.g., AAPL, MSFT, BTC-USD)."] = "AAPL"
    document_type: Annotated[str, "The document type related to user query (e.g., Form 10-K, Form 10-Q, Form 8-K)."] = "Form 10-K"
    year: Annotated[str, "The year(format: YYYY) or quarter of the year(format: YYYY QX) related to user query (e.g., 2015, 2001, 2017 Q3)."] = "2023"


In [30]:
def get_jsonparsed_data(url: str) -> dict:
    """
    Fetches and parses JSON data from a given URL.

    Args:
        url (str): The URL to fetch data from.

    Returns:
        dict: The JSON data as a dictionary.
    """
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)

def is_within_quarter(date_str: str, year: str, start_month: int, end_month: int) -> bool:
    """
    Checks if the given date falls within the specified fiscal quarter.

    Args:
        date_str (str): The date in 'YYYY-MM' format.
        year (str): The fiscal year to check.
        start_month (int): The starting month of the quarter.
        end_month (int): The ending month of the quarter.

    Returns:
        bool: True if the date is within the fiscal quarter, False otherwise.
    """
    date_parts = date_str.split('-')
    date_year = date_parts[0]
    date_month = int(date_parts[1])
    return date_year == year and start_month <= date_month <= end_month

def which_fiscal_year(date_str: str, fiscal_year: str) -> str:
    """
    Determines the fiscal year for a given date.

    Args:
        date_str (str): The filing date in 'YYYY-MM-DD' format.
        fiscal_year (str): The initial fiscal year.

    Returns:
        str: The fiscal year adjusted for the filing date.
    """
    filing_date = datetime.strptime(date_str.split()[0], "%Y-%m-%d")
    filing_month = filing_date.month
    fiscal_year = int(fiscal_year)

    if filing_month <= 3:
        fiscal_year += 1
    
    return str(fiscal_year)

def document_selector_downloader(
    ticker: Annotated[str, "The ticker symbol related to user query (e.g., AAPL, MSFT, BTC-USD)."] = "AAPL",
    document_type: Annotated[str, "The document type related to user query (e.g., Form 10-K, Form 10-Q, Form 8-K)."] = "Form 10-K",
    year: Annotated[str, "The year(format: YYYY) or quarter of the year(format: YYYY QX) related to user query (e.g., 2015, 2001, 2017 Q3)."] = "2023"
) -> None:
    """
    Downloads and saves a specific SEC document based on the provided parameters.

    Args:
        ticker (str): The ticker symbol of the company.
        document_type (str): The type of document to download (e.g., Form 10-K, Form 10-Q).
        year (str): The year or fiscal quarter to filter documents.
    
    Returns:
        None
    """
    pipeline_output = {'ticker': ticker, 'document_type': document_type, 'year': year}

    api_key = os.getenv('FMP_API_KEY')
    if api_key is None:
        raise ValueError("No API key found. Please set the FMP_API_KEY environment variable.")

    document_type_mapping = {
        "Form 10-K": "10-k",
        "Form 10-Q": "10-q"
    }

    quarter_map = {
        "Q1": (1, 3),
        "Q2": (4, 6),
        "Q3": (7, 9),
        "Q4": (10, 12)
    }

    filing_type = document_type_mapping.get(pipeline_output.get('document_type'))
    if not filing_type:
        raise ValueError(f"Unsupported document type: {pipeline_output.get('document_type')}")

    ticker = pipeline_output.get('ticker')
    if not ticker:
        raise ValueError("Ticker is required.")

    year_quarter = pipeline_output.get('year')
    if not year_quarter:
        raise ValueError("Year/Quarter is required.")
    
    if filing_type == "10-k":
        output_dir = os.path.join("data", "sec-edgar-filings", ticker, filing_type, year_quarter)
    elif filing_type == "10-q":
        year, quarter = year_quarter.split()
        start_month, end_month = quarter_map[quarter]
        output_dir = os.path.join("data", "sec-edgar-filings", ticker, filing_type, year, quarter)

    pdf_path = os.path.join(output_dir, 'primary_document.pdf')

    if os.path.exists(pdf_path):
        logging.info(f"PDF already exists at {pdf_path}. Skipping download.")
        return

    url = f"https://financialmodelingprep.com/api/v3/sec_filings/{ticker}?type={filing_type}&page=0&apikey={api_key}"

    try:
        fmp_results = get_jsonparsed_data(url)
    except Exception as e:
        raise RuntimeError(f"Failed to fetch data from FMP: {e}")

    if filing_type == "10-k":
        year_quarter = which_fiscal_year(fmp_results[0]['fillingDate'], year_quarter)
        filtered_fmp_results = [i for i in fmp_results if re.search(year_quarter, i['fillingDate'])]
        print(filtered_fmp_results)
        output_dir = os.path.join("data", "sec-edgar-filings", ticker, filing_type, year)
    elif filing_type == "10-q":
        year, quarter = year_quarter.split()
        start_month, end_month = quarter_map[quarter]
        filtered_fmp_results = [i for i in fmp_results if is_within_quarter(i['fillingDate'], year, start_month, end_month)]
        print(filtered_fmp_results)
        output_dir = os.path.join("data", "sec-edgar-filings", ticker, filing_type, year, quarter)

    if not filtered_fmp_results:
        raise ValueError("No matching SEC filings found.")

    os.makedirs(output_dir, exist_ok=True)

    path_to_wkhtmltopdf = os.getenv('WKHTMLTOPDF_PATH', r'C:\Program Files\wkhtmltopdf\bin\wkhtmltopdf.exe')
    if not os.path.exists(path_to_wkhtmltopdf):
        raise FileNotFoundError(f"wkhtmltopdf not found at {path_to_wkhtmltopdf}")

    url = filtered_fmp_results[0]['finalLink']
    pdf_path = os.path.join(output_dir, 'primary_document.pdf')

    config = pdfkit.configuration(wkhtmltopdf=path_to_wkhtmltopdf)

    try:
        pdfkit.from_url(url, pdf_path, configuration=config)
        logging.info(f"PDF generated and saved at {pdf_path}")
    except Exception as e:
        logging.error(f"PDF generation failed: {e}")
        raise


In [31]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from typing import List, Annotated

# Define the RAG-Fusion prompt template
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

# Define the pipeline for generating queries
generate_queries = (
    prompt_rag_fusion 
    | ChatOpenAI(temperature=0) 
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

def query_translation_rag_fusion(
    input: Annotated[str, "User input prompt or query related to qualitative data to undergo query translation"]
) -> List[str]:
    """
    Generates multiple search queries based on a single input prompt using the RAG-Fusion approach.

    Args:
        input (str): The user's input prompt or query for generating related search queries.

    Returns:
        List[str]: A list of generated search queries.
    """
    # Execute the generate_queries pipeline with the input
    pipeline = generate_queries
    queries = pipeline.invoke({"question": input})
    return queries


In [32]:
query_translation_rag_fusion("What are the biggest discussed risks for Amazon.com Inc. (AMZN)?")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


['1. What are the biggest cybersecurity risks for Amazon.com Inc. (AMZN)?',
 '2. What are the biggest regulatory risks for Amazon.com Inc. (AMZN)?',
 '3. What are the biggest competition risks for Amazon.com Inc. (AMZN)?',
 '4. What are the biggest supply chain risks for Amazon.com Inc. (AMZN)?']

In [33]:
system_message = """You are an expert financial analyst that always answers questions with the most relevant information using the tools at your disposal.
These tools have information regarding companies that the user has expressed interest in.
Here are some guidelines that you must follow:
* For financial questions, you must use the tools to find the answer and then write a response.
* Even if it seems like your tools won't be able to answer the question, you must still use them to find the most relevant information and insights. Not using them will appear as if you are not doing your job.
* You may assume that the users financial questions are related to the documents they've selected.
* For any user message that isn't related to financial analysis, respectfully decline to respond and suggest that the user ask a relevant question.
* If your tools are unable to find an answer, you should say that you haven't found an answer but still relay any useful information the tools found.

Provide a comprehensive answer to the user's question using the retrieved data. Dont just list data. If possible try to analyze the data and give interesting insights.

Return 'TERMINATE' when the task is completed.
"""

# Define the assistant agent
assistant = ConversableAgent(
    name="Assistant",
    system_message=system_message,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    max_consecutive_auto_reply=5,
    llm_config={"config_list": [{"model": "gpt-3.5-turbo-1106", "api_key": OPENAI_API_KEY}]},
    human_input_mode="NEVER",
)

# Define the user proxy agent
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)


In [36]:
register_function(
    query_translation_rag_fusion,
    caller=assistant,
    executor=user_proxy,
    name="query_translation_rag_fusion",
    description="This function helps in query translation of a user prompt related to qualitative financial data. Run this tool before running 'document_selector_downloader' tool and this is the step 1 of rag pipeline for answering questions related to qualitative financial data."
)

register_function(
    document_selector_downloader,
    caller=assistant,
    executor=user_proxy,
    name="document_selector_downloader",
    description="This function downloads specific financial documents (e.g., Form 10-K, Form 10-Q) for a given company ticker symbol and year or quarter. It fetches the documents from the Financial Modeling Prep API, filters the results based on the user's query, and saves the primary document as a PDF. The function requires a valid FMP API key and the wkhtmltopdf tool. This is the step 2 of rag pipeline for answering questions related to qualitative financial data."
)


In [37]:
chat_result = user_proxy.initiate_chat(
    assistant,
    message="What are the biggest discussed risks for Amazon.com Inc. (AMZN)?"
)

User (to Assistant):

What are the biggest discussed risks for Amazon.com Inc. (AMZN)?

--------------------------------------------------------------------------------


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Assistant (to User):

***** Suggested tool call (call_FY7HTzZYHaJ5GCgwLUfk46bX): document_selector_downloader *****
Arguments: 
{"ticker":"AMZN","document_type":"Form 10-K","year":"2022"}
*********************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION document_selector_downloader...


INFO:root:PDF already exists at data\sec-edgar-filings\AMZN\10-k\2022\primary_document.pdf. Skipping download.


User (to Assistant):

User (to Assistant):

***** Response from calling tool (call_FY7HTzZYHaJ5GCgwLUfk46bX) *****
null
**********************************************************************

--------------------------------------------------------------------------------


...

Assistant (to User):

It seems like your message was cut off. If you have a financial question or if there's anything specific you'd like to know, feel free to ask!

--------------------------------------------------------------------------------
User (to Assistant):



--------------------------------------------------------------------------------


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Assistant (to User):

It looks like the message was not completed. If you have a financial question or need assistance with anything else, please let me know how I can help!

--------------------------------------------------------------------------------
User (to Assistant):



--------------------------------------------------------------------------------


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Assistant (to User):

If you have any financial questions or need assistance with financial analysis, feel free to ask!

--------------------------------------------------------------------------------
User (to Assistant):



--------------------------------------------------------------------------------


In [8]:
chat_result = user_proxy.initiate_chat(
    assistant,
    message="What are the strengths and weakness of NVDA and AMD over the past two years?"
)

User (to Assistant):

What are the strengths and weakness of NVDA and AMD over the past two years?

--------------------------------------------------------------------------------
Assistant (to User):

***** Suggested tool call (call_UQkfsmWWIJ0AjnOOB3jIW9CP): document_selector_downloader *****
Arguments: 
{"ticker": "NVDA", "document_type": "Form 10-K", "year": "2020"}
*********************************************************************************************
***** Suggested tool call (call_CK6YbEWLIgcapkAKa5zuCDIe): document_selector_downloader *****
Arguments: 
{"ticker": "AMD", "document_type": "Form 10-K", "year": "2020"}
*********************************************************************************************
***** Suggested tool call (call_IAdWwBjLwICU0P5LFuwxf804): document_selector_downloader *****
Arguments: 
{"ticker": "NVDA", "document_type": "Form 10-K", "year": "2019"}
*********************************************************************************************
***

INFO:root:PDF already exists at data\sec-edgar-filings\NVDA\10-k\2020\primary_document.pdf. Skipping download.



>>>>>>>> EXECUTING FUNCTION document_selector_downloader...


INFO:root:PDF already exists at data\sec-edgar-filings\AMD\10-k\2020\primary_document.pdf. Skipping download.



>>>>>>>> EXECUTING FUNCTION document_selector_downloader...


INFO:root:PDF already exists at data\sec-edgar-filings\NVDA\10-k\2019\primary_document.pdf. Skipping download.



>>>>>>>> EXECUTING FUNCTION document_selector_downloader...


INFO:root:PDF already exists at data\sec-edgar-filings\AMD\10-k\2019\primary_document.pdf. Skipping download.


User (to Assistant):

User (to Assistant):

***** Response from calling tool (call_UQkfsmWWIJ0AjnOOB3jIW9CP) *****
null
**********************************************************************

--------------------------------------------------------------------------------
User (to Assistant):

***** Response from calling tool (call_CK6YbEWLIgcapkAKa5zuCDIe) *****
null
**********************************************************************

--------------------------------------------------------------------------------
User (to Assistant):

***** Response from calling tool (call_IAdWwBjLwICU0P5LFuwxf804) *****
null
**********************************************************************

--------------------------------------------------------------------------------
User (to Assistant):

***** Response from calling tool (call_1Nh4jn8w9XCvk8E4AdUZ3gLR) *****
null
**********************************************************************

-------------------------------------------------------

In [9]:
chat_result = user_proxy.initiate_chat(
    assistant,
    message="How did microsoft performing for past three years?"
)

User (to Assistant):

How is microsoft performing for past three years?

--------------------------------------------------------------------------------


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Assistant (to User):

***** Suggested tool call (call_bSauheYVRfYiae7Lwfl1cacK): document_selector_downloader *****
Arguments: 
{"ticker":"MSFT","document_type":"Form 10-K","year":"2019"}
*********************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION document_selector_downloader...


C:\Users\goldr\AppData\Local\Temp\ipykernel_17724\3999682696.py:2: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


[{'symbol': 'MSFT', 'fillingDate': '2019-08-01 00:00:00', 'acceptedDate': '2019-08-01 16:09:55', 'cik': '0000789019', 'type': '10-K', 'link': 'https://www.sec.gov/Archives/edgar/data/789019/000156459019027952/0001564590-19-027952-index.htm', 'finalLink': 'https://www.sec.gov/Archives/edgar/data/789019/000156459019027952/msft-10k_20190630.htm'}]


...

User (to Assistant):

User (to Assistant):

***** Response from calling tool (call_86OjskFMGIanGQ1MS6o5EQD3) *****
null
**********************************************************************

--------------------------------------------------------------------------------


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Assistant (to User):

I'm currently unable to retrieve the Form 10-K documents for Microsoft for the past three years. Is there anything else I can assist you with?

--------------------------------------------------------------------------------
User (to Assistant):



--------------------------------------------------------------------------------


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Assistant (to User):

It seems like your message was cut off. If you have a financial question or need assistance with anything else, feel free to ask!

--------------------------------------------------------------------------------
User (to Assistant):



--------------------------------------------------------------------------------


In [8]:
chat_result = user_proxy.initiate_chat(
    assistant,
    message="How did microsoft perform for the year 2023?"
)

User (to Assistant):

How did microsoft perform for the year 2023?

--------------------------------------------------------------------------------


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Assistant (to User):

***** Suggested tool call (call_9h5q08dmJWWvoQ642qWDP45I): document_selector_downloader *****
Arguments: 
{"ticker":"MSFT","document_type":"Form 10-K","year":"2023"}
*********************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION document_selector_downloader...


C:\Users\goldr\AppData\Local\Temp\ipykernel_20364\1706336277.py:2: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


[{'symbol': 'MSFT', 'fillingDate': '2023-07-27 00:00:00', 'acceptedDate': '2023-07-27 16:01:56', 'cik': '0000789019', 'type': '10-K', 'link': 'https://www.sec.gov/Archives/edgar/data/789019/000095017023035122/0000950170-23-035122-index.htm', 'finalLink': 'https://www.sec.gov/Archives/edgar/data/789019/000095017023035122/msft-20230630.htm'}]


...

Assistant (to User):

It seems like your message was cut off. If you have a financial question or if there's anything else I can assist you with, feel free to ask!

--------------------------------------------------------------------------------
User (to Assistant):



--------------------------------------------------------------------------------


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Assistant (to User):

It seems like your message was cut off. If you have a financial question or if there's anything else I can assist you with, feel free to ask! If you'd like to know more about Microsoft's financial performance or any other financial analysis, please let me know and I'll be happy to help.

--------------------------------------------------------------------------------
User (to Assistant):



--------------------------------------------------------------------------------


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Assistant (to User):

It seems like your message was not completed. If you have a financial question or need assistance with anything else, feel free to ask and I'll be happy to help!

--------------------------------------------------------------------------------
User (to Assistant):



--------------------------------------------------------------------------------


In [9]:
chat_result = user_proxy.initiate_chat(
    assistant,
    message="Which is the latest financial year?"
)

User (to Assistant):

Which is the latest financial year?

--------------------------------------------------------------------------------


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Assistant (to User):

***** Suggested tool call (call_s7UOMqzLOij4hjMzLOPu8VLL): document_selector_downloader *****
Arguments: 
{"ticker":"AAPL"}
*********************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION document_selector_downloader...


C:\Users\goldr\AppData\Local\Temp\ipykernel_20364\1706336277.py:2: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


[{'symbol': 'AAPL', 'fillingDate': '2023-11-03 00:00:00', 'acceptedDate': '2023-11-02 18:08:27', 'cik': '0000320193', 'type': '10-K', 'link': 'https://www.sec.gov/Archives/edgar/data/320193/000032019323000106/0000320193-23-000106-index.htm', 'finalLink': 'https://www.sec.gov/Archives/edgar/data/320193/000032019323000106/aapl-20230930.htm'}]


...

User (to Assistant):

User (to Assistant):

***** Response from calling tool (call_zfhmohDEe7HZZbhFZnKLi4Xf) *****
null
**********************************************************************

--------------------------------------------------------------------------------


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Assistant (to User):

I was unable to retrieve the latest Form 10-K for AAPL at this time. It's possible that the latest filing is not available yet, or there may be some technical difficulties in accessing the document. I would recommend checking the SEC's EDGAR database directly to access the latest Form 10-K filing for AAPL.

--------------------------------------------------------------------------------
User (to Assistant):



--------------------------------------------------------------------------------


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Assistant (to User):

It seems like your query is not related to financial analysis. If you have any financial questions or need assistance with data analysis, feel free to ask!

--------------------------------------------------------------------------------
User (to Assistant):



--------------------------------------------------------------------------------


In [3]:
import re
re.match(r"\d+", '800')[0]

'800'